In [1]:
from flask import Flask, render_template, request, redirect, url_for, session
from flask_mysqldb import MySQL
import MySQLdb.cursors
import re
import sqlite3

import os
os.getcwd()
loc=os.getcwd()+'\\templates'

app = Flask(__name__,template_folder=loc)
app.secret_key = 'xyzsdfg'

def get_db_connection():
    con = sqlite3.connect('DB_WEB.db')
    con.row_factory = sqlite3.Row
    return con

@app.route('/')
@app.route('/login', methods =['POST', 'GET'])
def login():
    mesage = ''
    if request.method == 'POST' and 'email' in request.form and 'password' in request.form:
        email = request.form['email']
        password = request.form['password']
        con = get_db_connection()
        cursor=con.cursor()
        cursor.execute('SELECT * FROM user1 WHERE email = ? AND password = ?', (email, password,))
        con.commit()
        user=cursor.fetchone()
        if user:
            session['loggedin'] = True
            session['name'] = user['name']
            session['email'] = user['email']
            mesage ='Logged in successfully !'
            name_filt=user['name']
            rows=cursor.execute('select * from exchange1').fetchall()
            rows1=cursor.execute('select * from exchange1 where name=?',(name_filt,)).fetchall()
            return render_template('user.html', rows=rows,entries=rows1,mesage=mesage)
        else:
            mesage = 'Please enter correct email / password !'
    return render_template('login.html', mesage = mesage)

@app.route('/', methods=['POST'])
def update():
    mesage = ''
    if request.method == 'POST' and 'name1' in request.form and 'date1' in request.form and 'count1' in request.form and 'exch1' in request.form and 'type1' in request.form and 'phone1' in request.form:
        in_name = request.form['name1']
        in_date = request.form['date1']
        in_count = request.form['count1']
        in_exch = request.form['exch1']
        in_type = request.form['type1']
        in_phone = request.form['phone1']
        con = get_db_connection()
        cursor  =  con.cursor()
        if 'save_ex' in request.form:
            cursor.execute("INSERT INTO exchange1 ('name','date','count','exch','type','phone') VALUES (?,?,?,?,?,?)", 
                           (in_name,in_date,in_count,in_exch,in_type,in_phone,))
        elif 'send_ex' in request.form:
            cursor.execute("DELETE from exchange1 where name = ? and date=? and count=? and exch=? and type=? and phone=?", 
                           (in_name,in_date,in_count,in_exch,in_type,in_phone,))
        con.commit()
        mesage = ''
    elif request.method == 'POST':
        mesage = ''
    return render_template('login.html',mesage=mesage)

@app.route('/logout')
def logout():
    session.pop('loggedin', None)
    session.pop('email', None)
    return redirect(url_for('login'))
  
@app.route('/register', methods =['POST', 'GET'])
def register():
    mesage = ''
    if request.method == 'POST' and 'name' in request.form and 'password' in request.form and 'email' in request.form :
        userName = request.form['name']
        password = request.form['password']
        email = request.form['email']
        con = get_db_connection()
        cursor  =  con.cursor()
        cursor.execute('SELECT * FROM user1 WHERE email = ?', (email,))
        con.commit()
        account = cursor.fetchone()
        if account:
            mesage = 'Account already exists !'
        elif not re.match(r'[^@]+@[^@]+\.[^@]+', email):
            mesage = 'Invalid email address !'
        elif not userName or not password or not email:
            mesage = 'Please fill out the form !'
        else:
            cursor.execute("INSERT INTO user1 ('name','email','password') VALUES ( ?, ?, ?)", (userName, email, password,))
            con.commit()
            mesage = 'You have successfully registered !'
    elif request.method == 'POST':
        mesage = 'Please fill out the form !'
    return render_template('register.html', mesage = mesage)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Feb/2023 16:43:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2023 16:43:56] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Feb/2023 16:44:01] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2023 16:44:02] "GET /logout HTTP/1.1" 302 -
127.0.0.1 - - [01/Feb/2023 16:44:02] "GET /login HTTP/1.1" 200 -
